<hr style="border-color:coral; border-width:2px">

## Using the Cartesian communicator

<hr style="border-color:coral; border-width:2px">

We can use a Cartesian communicator to impose a Cartesian grid structure on a set of processors. 

In [1]:
%matplotlib notebook
%pylab

Using matplotlib backend: nbAgg
Populating the interactive namespace from numpy and matplotlib


<hr style="border-color:coral; border-width:2px">

# Shift data and replace

In this example, each processor stores a single integer value, the rank of the processor.  We create a Cartesian communicator and then shift each value to a neighboring processor, depending on the value of the displacement.  

* If **displacement** is 1, each processor sends its value to its right neighbor

* If **displacement** is -1, each processor sends its value to its left neighbor

If the domain is **periodic**, each processor has a neighbor.  If the domain is not periodic, then the left most processor does not have a left neighbor, and the rightmost processor does not have a right neighbor. 

In [17]:
%%file cart_shift.c

#include <mpi.h>
#include <math.h>

#include <stdio.h>
#include <stdlib.h>

void main(int argc, char** argv)
{
    MPI_Init(&argc, &argv);

    int rank, nprocs;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &nprocs);
    
    if (argc < 2)
    {
        if (rank == 0)
        {
            printf("Usage : \n");
            printf("    cart_shift <disp> <periodicity>\n");
        }
        exit(0);
    }

    /* Read input values */
    int disp = atoi(argv[1]);
    int p = atoi(argv[2]);

    /* Each processor stores a single integer, equal to its rank */
    int myval = rank;

    MPI_Comm comm_cart;  /* Create a new communicator */
    int ndim = 1;
    int dims[1] = {nprocs};
    int periodicity[1] = {p};
    int reorder = 0;
    MPI_Cart_create(MPI_COMM_WORLD, ndim, dims,  periodicity, reorder, &comm_cart);

    /* source : This proc receives data from SOURCE
       dest   : This proc sends data to DESTINATION */
    
    int dir = 0;
    int source, dest;
    MPI_Cart_shift(comm_cart, dir, disp, &source, &dest);
    
    /* Check for left and right boundaries */
    if (source == MPI_PROC_NULL && disp == 1)
    {
        printf("Rank %d : No left neighbor found\n",rank);
    }
    else if (source == MPI_PROC_NULL && disp == -1)
    {
        printf("Rank %d : No right neighbor found\n",rank);
    }

    int tag = 0;
    MPI_Sendrecv_replace(&myval, 1, MPI_INTEGER, dest, tag, 
                                    source, tag, comm_cart, MPI_STATUS_IGNORE);

    printf("Rank %d : myval = %d (%d, %d)\n",rank, myval, source, dest);

    MPI_Finalize();
}

Overwriting cart_boundary_exchange.c


In [19]:
%%bash 

mpicc -o cart_shift cart_shift.c

mpirun -n 4 cart_shift -1 1

Rank 2 : myval = 3 (3, 1)
Rank 3 : myval = 0 (0, 2)
Rank 0 : myval = 1 (1, 3)
Rank 1 : myval = 2 (2, 0)


<hr style="border-color:coral; border-width:2px">

# Filling ghost cell values

<hr style="border-color:coral; border-width:2px">

We can also use the Cartesian communicator to fill ghost cell values.  

In [27]:
%%file cart_fill_ghost.c

#include <mpi.h>
#include <math.h>

#include <stdio.h>
#include <stdlib.h>

void main(int argc, char** argv)
{
    MPI_Init(&argc, &argv);

    int rank, nprocs;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &nprocs);
    
    if (argc == 1)
    {
        if (rank == 0)
        {
            printf("Usage : \n");
            printf("    cart_fill_ghost <periodicity>\n");
        }
        exit(0);
    }

    /* Read input values */
    int p = atoi(argv[1]);
    
    double qmem[3];    
    double *q = &qmem[1];

    q[-1] = -999.;
    q[0] = rank;
    q[1] = 999;

    /* Fill in q[-1] and q[1] with data from neighbors */
    MPI_Comm comm_cart;
    int ndim = 1;
    int dims[1] = {nprocs};
    int periodicity[1] = {p};
    int reorder = 0;
    MPI_Cart_create(MPI_COMM_WORLD, ndim, dims,  periodicity, reorder, &comm_cart);

    /* Fill left ghost cells */
    int dir = 0;
    int disp = 1;
    int source, dest;
    int ierr = MPI_Cart_shift(comm_cart, dir, disp, &source, &dest);

    int tag = 0;
    MPI_Sendrecv(&q[0], 1, MPI_DOUBLE, dest, tag, 
                            &q[-1], 1, MPI_DOUBLE, source, tag, comm_cart, MPI_STATUS_IGNORE);

    /* Fill right ghost cells */
    disp = -1;  
    MPI_Cart_shift(comm_cart, dir, disp, &source, &dest);

    tag = 1;
    ierr = MPI_Sendrecv(&q[0], 1, MPI_DOUBLE, dest, tag, 
                        &q[1], 1, MPI_DOUBLE, source, tag, comm_cart, MPI_STATUS_IGNORE);

    printf("Rank %d : (%g, %g, %g)\n",rank, q[-1], q[0], q[1]);

    MPI_Finalize();
}

Overwriting cart_fill_ghost.c


In [31]:
%%bash 

mpicc -o cart_fill_ghost cart_fill_ghost.c

mpirun -n 7 cart_fill_ghost 0

Rank 0 : (-999, 0, 1)
Rank 1 : (0, 1, 2)
Rank 2 : (1, 2, 3)
Rank 3 : (2, 3, 4)
Rank 4 : (3, 4, 5)
Rank 5 : (4, 5, 6)
Rank 6 : (5, 6, 999)
